# Roadmap

$$ \text{Raster}\xleftarrow{\text{1ms bin}} \text{Relative Response: } T \times (N * B)$$
<br>
$$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\downarrow{\frac{\Sigma{\text{ (columns: T)}}} {\text{T in E}}} $$
<br>
$$ PSTH $$
<br>
$$\text{T = trials | N = neurons | B = bins | E = events}$$

# Goal
* Find neurons' relative response to each trial
* Create the relative response matrix: Trials×(Neurons*Total Bins)
  * Create raster plot if bin size = 1ms
  * Sum columns and create PSTH
    * Plot PSTHS
    * Receptive field analysis if bin size = 5ms

# JSON Example
```json
{
    "events": {
        "event_1": [
            104.6154
        ],
         "event_4": [
            42.74935
        ]
    }
}
```

# Window of Interest
* pre_time: Amount of time taken before event onset
  * ex: -0.2s (200ms before event onset)
* post_time: Amount of time taken after event onset
  * ex: 0.2s (200ms after event onset)
* bin_size: size of step to group spikes for a given neuron
  * ex: 0.1s (100ms bin to group spikes into)

# Finding Spikes Relative to Trial
$$\text{Neuron Spike Timestamps} - \text{Trial Timestamp} = \text{Relative Spike Timestamps}$$
<br>
$$\begin{bmatrix}103.3154\\104.4796\\104.6479\\104.6178\\104.7277\\104.7632\\104.7796\\104.8918\end{bmatrix}-104.6154=\begin{bmatrix}-1.3\\-0.1358\\0.0024\\0.0325\\0.1123\\0.1478\\0.1642\\0.2764\end{bmatrix}$$


# Binning Spikes
$$\mathrm{pre\_time} = -0.2, \mathrm{post\_time} = 0.2, \mathrm{bin\_size} = 0.1$$
<br>
$$\text{Bin edges} = [-0.2, -0.1), [-0.1, 0), [0, 0.1), [0.1, 0.2]$$
$$\text{Note that both edges are not inclusive so that if a spike does fall on an edge it is not counted twice}$$
<br>
$$\begin{bmatrix}-1.3\\-0.1358\\0.0024\\0.0325\\0.1123\\0.1478\\0.1642\\0.2764\end{bmatrix}\xrightarrow{\text{histogram function call}}\begin{bmatrix}1\\0\\2\\3\end{bmatrix}$$


# Relative Response Matrix
The relative response matrix has dimensions trials (rows) by bins (columns) for a given neuron as seen below
$$\text{Event 1, Neuron 1}$$
$$\begin{bmatrix}trial 1\\trial 2\\trial 3\end{bmatrix}\begin{bmatrix}0,0,3,4\\1,0,2,3\\0,1,5,2\end{bmatrix}$$
## Calculating PSTH
$$\begin{bmatrix}0,0,3,4\\1,0,2,3\\0,1,5,2\end{bmatrix}\xrightarrow{\text{sum spikes across trials}}\begin{bmatrix}1,1,10,9\end{bmatrix}\xrightarrow{\text{divide by total trials (3)}}\begin{bmatrix}0.33,0.33,3.33,3\end{bmatrix}$$

# Example PSTH
![Sig 007c Event 4 PSTH](images/sig007c_event_4_0.01ms_psth.png)

# Receptive Field Calculations
$$psth = \begin{bmatrix}0.33,0.33,3.33,3\end{bmatrix}$$
1. split psth on event onset to get the pre and post event activity separated
$$\mathrm{pre\_post} = [0.33, 0.33]$$
$$\mathrm{post\_psth} = [3.33, 3]$$
2. calculate background firing rate and threshold
  * note std = standard deviation
```py
std_scalar = 3 # scales the number of standard deviations
background_firing_rate = mean(pre_psth)
threshold = background_firing_rate + std_scalar * std(pre_psth)
```
3. Apply threshold to post psth to get latencies
```py
above_threshold = post_psth > threshold # returns boolean array where values > threshold
first_bin = above_threshold[0] # grab the first bin above the threshold
last_bin = above_threshold[-1] # grab the last bin above threshold
first_bin_latency = first_bin * bin_size # have to scale bin index to time
last_bin_latency = last_bin * bin_size
[peak, peak_index] = max(post_psth)
peak_latency = peak_index * bin_size
```
4. Calculate Response Magnitude
```py
response_magnitude = sum(post_psth[first_bin:last_bin + 1]) #+1 since python indexing is non inclusive
```

# Example PSTH with Receptive Field
![Sig 007c Event 4 PSTH](images/sig007c_event_4_0.005ms_psth.png)

# Matlab Histogram Function Call
```py
pre_time = -0.2; post_time = 0.2; bin_size = 0.1;
event_window = pre_time:bin_size:post_time;
# relative_spikes is the offset spike times for a given trial
[binned_spikes, ~] = histcounts(relative_spikes, event_window);
```
# Numpy Histogram Function Call
```py
import numpy as np

pre_time = -0.2; post_time = 0.2; bin_size = 0.1;
event_window = list(np.arange(pre_time, post_time, bin_size))
total_bins = len(event_window)
# relative_spikes is the offset spike times for a given trial
# np.histogram returns an array [histogram, bin_edges] so the call below only grabs the histogram
binned_spikes = np.histogram(relative_spikes, total_bins, range = (pre_time, post_time))[0]
```

# What to turn in
## Conceptual Questions and graphs (10pts)
* pdf with answers to conceptual questions and requested psths and raster graphs
* Code files you wrote to create JSON output file
* Code files should be plain text with proper file extension
  * ex: last_first_hw1.m, last_first_hw1.py, etc.

# Json Output File (10pts)
* This file will be input into automatic grader
* This was taken from the example solution json and is a small snippet
* You are not required to turn in a pretty formatted json file
```json
{
    "event_1": {
        "sig001a": {
            "background_rate": 0.013732394366197184,
            "first_bin_latency": 0.0925,
            "last_bin_latency": 0.1325
        }
    }
}
```

# Python Raster Sidenote
## Example
Parameters: 1ms bin size from 0ms to 5ms
$$\text{0 0 1 0 1} \xrightarrow{\text{transformation}} \text{NaN NaN 0.003 NaN 0.005}$$
### Mapping
$$0 \rightarrow NaN$$
$$1 \rightarrow \text{scaled time point}$$

# Sig007c Event 4 Raster
Y-axis: trials X-axis: Time (s)
![Sig 007c Event 4 Raster](images/sig007c_event_4_0.001ms_raster.png)